In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import to_datetime
from pandas import DataFrame
from fbprophet import Prophet
from matplotlib import pyplot
from pandas import read_csv
from pandas import to_datetime
from pandas import DataFrame
from fbprophet import Prophet
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

Importing plotly failed. Interactive plots will not work.


In [ ]:
df = read_csv('Data/service_request_compare/sr_hex.csv')

In [ ]:
df2 = df.loc[((df['department'] == 'Water and Sanitation'))]

In [ ]:
df2 = df2[['CreationTimestamp','h3_level8_index']]

In [ ]:
df2.head()

In [ ]:
df2['CreationTimestamp'] = df2.loc[:, ('CreationTimestamp')].astype(str).str[:10]

In [ ]:
df2['h3_level8_index'] = df2['h3_level8_index'].astype(str)

In [ ]:
df2.insert(loc=2, column='Total_Requests',value=1)

In [ ]:
df2['CreationTimestamp'] = pd.to_datetime(df2['CreationTimestamp'])

In [ ]:
df2 = df2.groupby(['h3_level8_index', pd.Grouper(key='CreationTimestamp', freq='W-WED')])['Total_Requests'].sum().reset_index().sort_values('CreationTimestamp')
print (df2)

In [ ]:
df3 = df2.loc[~((df2['h3_level8_index'] == '0'))]

In [ ]:
df3 = df3.groupby(by=['CreationTimestamp']).sum('Total_Requests').reset_index()

In [ ]:
df3.tail()

In [ ]:
# prepare expected column names
df3.columns = ['ds', 'y']

In [ ]:
#New Forecasts

In [ ]:
#df4 = df3

In [ ]:
#model_2 = Prophet(interval_width = 0.95, weekly_seasonality=True)
#model = model_2.fit(df4)

In [ ]:
#future = model_2.make_future_dataframe(periods=4, freq='W')
#forecast = model_2.predict(future)

In [ ]:
#forecast.tail(10)

In [ ]:
#df4.tail(10)

In [ ]:
#model_2.plot(forecast)

In [ ]:
#len(df4)

In [ ]:
#New Forecasts

In [ ]:
#train=df4[:138]
#test=df4[138:]

#m=Prophet(weekly_seasonality=True)
#m.fit(train)
#future=m.make_future_dataframe(periods=40,freq='W')
#forecast=m.predict(future)


In [ ]:
#forecast.taill()

In [ ]:
#df4.tail()

In [ ]:
#m.plot(forecast, uncertainty=True)
#plt.show()

In [ ]:
#m.plot(forecast)
#ax=forecast.plot(x='ds',y='yhat',legend=True,label='predictions',figsize=(12,8))
#test.plot(x='ds',y='y',legend=True,label='True Test Data',ax=ax,xlim=('2017-01-02','2020-01-06'))

In [ ]:
#train, test = train_test_split(df4, test_size = 0.10, shuffle=False)
#model_fit = model_2.fit (train)
#future = model_.make_future_dataframe(periods=20, freq='W')
#predictions = model_fit.predict(future)
#model_fit.plot(predictions)

In [ ]:
#predictions.tail()

In [ ]:
#df3.tail()

In [ ]:
model = Prophet(weekly_seasonality=True)

train, test = train_test_split(df3, test_size = 0.10, shuffle=False)
model_fit = model.fit (train)


In [ ]:
future = model_fit.make_future_dataframe(periods=20, freq='W')
predictions = model_fit.predict(future)

In [ ]:
model_fit.plot(predictions)

In [ ]:
model_fit.plot_components(predictions)

In [ ]:
predictions['yhat'] = np.round(predictions['yhat'])
final = predictions.rename({'ds': 'date','yhat' : 'Service Requests Expected'}, axis=1)

In [ ]:
print("Final Answer:" )
final[['date','Service Requests Expected']].tail()


In [ ]:
df3.tail()